In [17]:
import simulation as sim
import mass_sampling_functions_fin as sim2
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
import pandas as pd

#Load Cosmology
omega_m0, omega_b0, cosmo_h, sigma8, ns = sim.cosmo_para()

#Prepare directory to store simulated images
cwd = os.getcwd()
directory = 'test'
path = os.path.join(cwd, directory)
shutil.rmtree(path)
os.mkdir(path)

#Select options for images
# nums = np.arange(9) + 1
nums = [8,9]


# default inputs
mass_range = (14.0, 16.0)
redshift = 0.0 #float ex for func A 
#redshift = (0.0, 1.5) #tuple ex for func B
sample_num = 100000
mdef = '200c'
model = 'bocquet16'

if type(redshift) is float: 
    mass_sample = np.array([sim2.mass_sampling(mass_range, redshift, mdef, model, sample_num)])

else:
    rs_dist = "skewnorm"
    mass_sample = []
    tot_num, filepath, df = sim2.mul_redshift_mass_sampling(rs_dist, rs_range, mass_range, mdef, model, sample_num, store = True)
    df_arr = df.to_numpy()

    
#Select Telescope and Observation Frequency
tele = 'SPT'
f = 150

#Retrieve Observation-related Parameters
beam_size, noise_level = sim.tele_para(tele, f)

#Converting to correct scale
f *= 1e9

for i in range(tot_num):
    z = df_arr[i, 1]
    mass = df_arr[i, 0]
    
    radius = np.linspace(0.1, 9, 10)     #arcmin
    new_r = sim.arcmin_to_Mpc(radius, z[i])
    battaglia_pro = np.array([sim.battaglia_profile(r, mass, z) for r in new_r])
    y_pro = sim.epp_to_y(battaglia_pro)
    
    abs_path = os.path.join(path, str(i))
    sim.generate_img(radius, y_pro, f, noise_level, beam_size, z[i], nums, p = abs_path, AP = True)

ModuleNotFoundError: No module named 'mass_sampling_functions_fin'